In [ ]:
#| hide
from hamux.core import *

# hamux

> **H**ierarchial **A**ssociative **M**emory **U**ser e**X**perience

This file will become your README and also the index of your documentation.

## Install

```sh
pip install hamux
```

## How to use

Fill me in please! Don't forget code examples: